In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime,timedelta
import time
from get_histdata import GetHistData_yf
from regret_calc import CalculateReturns,CalculateReturnsList
from recent_trends import recent_trends
from scan_breakouts import daily_scan

In [10]:
def GetHistData_yf(tickers,start=None,end=None,interval='1d'):
    '''This function retrieves historical data of the list of tickers provided.
    GetHistData_yf(tickers,start=None,end=None,interval='1d')
    ---------------------------------
    (tickers, list)                   = A list containg ticker symbols as listed yfinance.
                                        Ex. ['ADANIPORTS.NS','TCS.NS','LT.NS','AXISBANK.NS',]
    (start, datetime or str,optional) = Start date of historical data. default is 3 months from current date.
                                        ex. '2021-08-01' 
    (end, datetime or str,optional)   = End date of historical data. default is current date.
                                        ex. '2022-08-01'
    (interval, string)                = The price interval. default is 1 day.
                                        ex. '1d', '1h', '5m'
    '''
    
    if start==None:
        from_date=(datetime.today()-timedelta(90)).strftime("%Y-%m-%d")
    elif type(start) == str:
        from_date=datetime.strptime(start,"%Y-%m-%d")
    else:
        from_date=datetime.strptime(start.strftime("%Y-%m-%d"),"%Y-%m-%d")     
    
    if end==None:
        to_date=datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d") 
    elif type(end) == str:
        to_date=datetime.strptime(end,"%Y-%m-%d")
    else:
        to_date=end

        
    start_time = time.time()
    hist_data=pd.DataFrame()
    # Get historical data
    print("Getting historical using yfinance")
    print("Fetching data from {} to {}".format(from_date,to_date))

    for i,tick in enumerate(tickers):
        security=yf.Ticker(tick)
    #     print(i,tick)
    #     print(security.splits)
        if i == 0 :
            hist_data=security.history(start=from_date,end=to_date,interval=interval)
            hist_data['Symbol']=tick
        else :
            df2=security.history(start=from_date,end=to_date,interval=interval)
            if not df2.empty:
                df2['Symbol']=tick
                hist_data=pd.concat([hist_data,df2])

    print("--- %s seconds ---" % (time.time() - start_time))
    return hist_data

In [10]:
data=pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1]
tickers=data['Symbol'].to_list()
tickers=["{}.NS".format(tick) for tick in tickers]
tickers.append('^NSEI')
tickers
df1=GetHistData_yf(tickers)

Getting historical using yfinance
Fetching data from 2022-03-14 00:00:00 to 2022-06-12 00:00:00
--- 17.965324878692627 seconds ---


In [11]:
df1

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2022-03-14,725.950012,733.500000,724.000000,730.549988,1905433,0.0,0,ADANIPORTS.NS
2022-03-15,733.000000,736.000000,716.250000,721.299988,4228582,0.0,0,ADANIPORTS.NS
2022-03-16,726.000000,729.500000,721.000000,723.799988,2693057,0.0,0,ADANIPORTS.NS
2022-03-17,730.500000,743.799988,729.549988,740.099976,4957379,0.0,0,ADANIPORTS.NS
2022-03-21,741.000000,750.450012,725.400024,735.150024,6662688,0.0,0,ADANIPORTS.NS
...,...,...,...,...,...,...,...,...
2022-06-06,16530.699219,16610.949219,16444.550781,16569.550781,233600,0.0,0,^NSEI
2022-06-07,16469.599609,16487.250000,16347.099609,16416.349609,233800,0.0,0,^NSEI
2022-06-08,16474.949219,16514.300781,16293.349609,16356.250000,243500,0.0,0,^NSEI


In [12]:
reg=CalculateReturnsList(df1['Symbol'].unique())

--- 18.01120901107788 seconds ---


In [20]:
reg1.to_excel (r'Reports/Nifty50_regret.xlsx', index = False, header=True)

In [7]:
sec=yf.Ticker('JUBLFOOD.NS')
sec.splits

Date
2018-06-21    2.0
2022-04-19    5.0
Name: Stock Splits, dtype: float64

In [19]:
reg1=reg.sort_values(by='Current_val', ascending=False)

In [16]:
reg

,Symbol,Shares,Entry_date,Exit_date,Entry_price,Exit_price,Invested,Current_val,Days_HODL,Percentage_chg
0,ADANIPORTS.NS,13.7,2022-03-14,2022-06-12,730.55,726.40,10000,9942.96,90,-0.57%
1,APOLLOHOSP.NS,2.0,2022-03-14,2022-06-12,4923.50,3700.50,10000,7515.72,90,-24.84%
2,ASIANPAINT.NS,3.4,2022-03-14,2022-06-12,2945.82,2708.75,10000,9196.21,90,-8.04%
3,AXISBANK.NS,14.1,2022-03-14,2022-06-12,710.05,660.70,10000,9305.30,90,-6.95%
4,BAJAJ-AUTO.NS,2.9,2022-03-14,2022-06-12,3452.55,3881.00,10000,11239.38,90,12.39%
5,BAJFINANCE.NS,1.5,2022-03-14,2022-06-12,6665.45,5667.50,10000,8501.25,90,-14.99%
6,BAJAJFINSV.NS,0.6,2022-03-14,2022-06-12,15678.50,12253.50,10000,7817.73,90,-21.82%
7,BHARTIARTL.NS,14.3,2022-03-14,2022-06-12,700.40,673.35,10000,9614.09,90,-3.86%
8,BPCL.NS,27.8,2022-03-14,2022-06-12,359.65,331.00,10000,9203.46,90,-7.97%
9,BRITANNIA.NS,3.1,2022-03-14,2022-06-12,3185.85,3360.85,10000,10549.71,90,5.5%
